In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
# misc
import os
from tqdm import tqdm, tnrange, tqdm_notebook
from pathlib import Path
import re
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sys
import scipy.ndimage
import math
import pandas as pd
#import PIL
from PIL import Image

In [4]:
# dicom and nii
import pydicom
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir
import pydicom.pixel_data_handlers.gdcm_handler as gdcm_handler
## ! gdcm must be installed with conda install (conda install -c conda-forge gdcm)
pydicom.config.image_handlers = ['gdcm_handler']
import nibabel as nib

In [5]:
# fast.ai lib
from fastai.vision import *
from fastai.metrics import *
from fastai.callbacks import *

In [6]:
# multithreading
from multiprocessing import Pool

## Define paths

In [7]:
path_str = '/home/ubuntu/sfr-challenge/lungs/dataset/Inference'

In [8]:
path = Path(path_str)

In [9]:
path_out = path/'Patients'
os.path.exists(path_out)

True

In [10]:
# UPDATE FOLDER NAME HERE
path_dcm = path/'Dataset-3'
os.path.exists(path_dcm)

True

## Define fixed random seed

In [11]:
np.random.seed(42)

## Define file loaders

In [37]:
# debug sorting https://pydicom.github.io/pydicom/dev/auto_examples/image_processing/reslice.html
# order reversed must be based on patients orientation / origin from the metadata
# ImagePositionPatient is used to sort slides
def load_dicom(path):
    slices = [pydicom.dcmread(path + '/' + s) for s in os.listdir(path) if ".dcm" in s.lower()]
    #try:
    #    slices.sort(key = lambda x: x.ImagePositionPatient[2], reverse=False)
    #except:
    slices.sort(key = lambda x: x.InstanceNumber, reverse=True)
        #print("No ImagePositionPatient[2] at:", path)
    return slices

In [13]:
#     except ValueError: can be used to 
def get_pixels_hu(slices, path):
    arr = np.array([], dtype=np.int16)
    # convert list of slices into ndarray
    try:
        arr = np.stack([s.pixel_array for s in slices])
    except:
        print('cannot stack at: ', path)
    arr = arr.astype(np.int16)
    # set outside-of-scann pixels to zero (200 by default)
    # see housfield units for more details https://en.wikipedia.org/wiki/Hounsfield_scale
    arr[arr == -2000] = 0
    # read Intercept and slope convert to Housfield units (HU)
    try:
        intercept = slices[0].RescaleIntercept
    except ValueError:
        print('cannot get intercept at: ', path)
    try:
        slope = slices[0].RescaleSlope
    except:
        print('cannot get slope at: ', path)
    if slope != 1:
        arr = slope * arr.astype(np.float64)
        arr = arr.astype(np.int16)
    arr += np.int16(intercept)
    return np.array(arr, dtype=np.int16)

## Below section: create RGB image from 3D numpy array

In [14]:
def get_rgb_image(label, img_3d, stride=1):
    step = stride + 1
    
    slice_n_minus_1 = img_3d[label - step]
    slice_n = img_3d[label]
    slice_n_plus_1 = img_3d[label + step]
    img = np.array([slice_n_minus_1,\
                    slice_n,\
                    slice_n_plus_1])
    return(np.moveaxis(img, 0, 2))

In [15]:
# normalize to [0; 65535]
# alternative normalization functions are in the "Test" section
MIN = -1000.0
MAX = 400.0
def normalize_cv2(image):
    image[image > MAX] = MAX
    image[image < MIN] = MIN
    #image = cv2.normalize(image, dst=None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    image = cv2.normalize(image, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    return image

In [16]:
# target depth is a target thickness in mm of the rgb x3 layers stack;
# It is basically the distance between exterior layers R and B
def get_stride(_slice):
    s = _slice.SliceThickness
    #print('SliceThickness:', s)
    if s >= 2 and s <= 3: stride = 0
    elif s > 0.9 and s <= 2: stride = 1
    elif s > 0.6 and s <= 0.9: stride = 2
    elif s > 0.5 and s <= 0.6: stride = 3
    else:
        print('SliceThickness is: ', s, ' at: ', _slice.PatientName)
        stride = 3
        print('stride out of range')
    #print('stride: ', stride)
    return int(stride)

## Image processing

In [17]:
def crop_center(img_nda, new_width=256, new_height=256):
    shape = img_nda.shape
    width = shape[0]
    height = shape[1]
    left = round((width - new_width) / 2)
    top = round((height - new_height) / 2)
    right = round((width + new_width) / 2)
    bottom = round((height + new_height) / 2)
    if (left > 0 and right > left and top > 0 and bottom > top): 
        crop_img = img_nda[top:bottom, left:right]
    else:
        crop_img = img_nda
    return (crop_img)

In [18]:
def resize(img, slc, new_spacing=[1.2,1.2]):
    init_spacing = [slc.PixelSpacing[0], slc.PixelSpacing[1]]
    spacing = map(float, init_spacing)
    spacing = np.array(list(spacing))
    
    resize_factor = spacing / new_spacing
    new_shape_float = img.shape[:2] * resize_factor
    new_shape = np.round(new_shape_float)
    
    resize_factor_real = new_shape / img.shape[:2]
    new_spacing = spacing / resize_factor_real
    img = cv2.resize(img, None, fx=resize_factor_real[0], fy=resize_factor_real[1])
    #img = scipy.ndimage.interpolation.zoom(img, np.append(resize_factor_real, [1]))
    return (img)

In [19]:
def split_quart(img, patient, qt_width=224, qt_height=224, pad_w=5, pad_h=5):
    shape = img.shape
    width = shape[0]
    height = shape[1]
    center_w = round(width / 2)
    center_h = round(height / 2)
    if center_h + pad_h - qt_height <= 0 or center_w + pad_w - qt_width <= 0:
        pad_w = qt_width - center_w
        pad_h = qt_height - center_h
        print('split_quart: image: ', patient, ' too small: ', shape)
    qt_upl = img[center_h + pad_h - qt_height : center_h + pad_h, center_w + pad_w - qt_width : center_w + pad_w]
    qt_upr = img[center_h + pad_h - qt_height : center_h + pad_h, center_w - pad_w : center_w - pad_w + qt_width]
    qt_dwl = img[center_h - pad_h : center_h - pad_h + qt_height, center_w + pad_w - qt_width : center_w + pad_w]
    qt_dwr = img[center_h - pad_h : center_h - pad_h + qt_height, center_w - pad_w : center_w - pad_w + qt_width]
    return qt_upl, qt_upr, qt_dwl, qt_dwr

## Export patients

In [30]:
patients = os.listdir(path_dcm)
len(patients)

344

### Export method 1: multi-process 2,7s/patient, 15 min total

In [22]:
def export_ds(p):
    #load dcm from patient folder
    slices = load_dicom(str(path_dcm/p))
    stride = get_stride(slices[0])
    step = 2 * (stride + 1)
    pad = step * 4
    #create patient folder in path_out if does't exist
    if not os.path.exists(path_out/p): os.makedirs(path_out/p)

    img_3d = get_pixels_hu(slices, path_dcm/p)
    cent_slice = pad
    num_slices = len(slices) - pad
    while cent_slice < num_slices:
        img_rgb = normalize_cv2(get_rgb_image(cent_slice, img_3d, stride=stride))
        img_rgb = resize(img_rgb, slices[0], new_spacing=[1.1, 1.1])
        img_rgb = crop_center(img_rgb)
        cv2.imwrite(str(path_out/p) + '/' + p + '_' + str(cent_slice) + '.tif', img_rgb)
        cent_slice += step
    return(str(path_out/p))

In [36]:
pats_bis = ['GSUuZtg', 'RFROAgY', 'RFROAgY']

In [ ]:
if  len(os.listdir('/home/varun/temp') :
    print("Directory is empty")
else:    
    print("Directory is not empty")  

In [42]:
exp = os.listdir(path_out)

In [49]:
# empty patients
empt_pats = []
for file in exp:
    if len(os.listdir(path_out/file)) == 0:
        empt_pats.append(subdir)

In [23]:
# CV2 has 4 threads each process. Maximal processing speed achieved with 2 process (8-core CPU)
prc = Pool(2)

No ImagePositionPatient[2] at: /home/ubuntu/sfr-challenge/lungs/dataset/Inference/Dataset-3/WTCRC14
cannot stack at:  /home/ubuntu/sfr-challenge/lungs/dataset/Inference/Dataset-3/WTCRC14
No ImagePositionPatient[2] at: /home/ubuntu/sfr-challenge/lungs/dataset/Inference/Dataset-3/QsaIA1D
cannot stack at:  /home/ubuntu/sfr-challenge/lungs/dataset/Inference/Dataset-3/QsaIA1D
No ImagePositionPatient[2] at: /home/ubuntu/sfr-challenge/lungs/dataset/Inference/Dataset-3/cD7Jiqn
cannot stack at:  /home/ubuntu/sfr-challenge/lungs/dataset/Inference/Dataset-3/cD7Jiqn
No ImagePositionPatient[2] at: /home/ubuntu/sfr-challenge/lungs/dataset/Inference/Dataset-3/rJWLLEG
cannot stack at:  /home/ubuntu/sfr-challenge/lungs/dataset/Inference/Dataset-3/rJWLLEG
No ImagePositionPatient[2] at: /home/ubuntu/sfr-challenge/lungs/dataset/Inference/Dataset-3/HjERXx3
cannot stack at:  /home/ubuntu/sfr-challenge/lungs/dataset/Inference/Dataset-3/HjERXx3


In [ ]:
%%time
prc.map(export_ds, patients)

### Export method 2: loop 5s/patient, 30 min total

In [ ]:
for p in tqdm_notebook(patients[:20]):
    export_ds(p)